<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_docs_yaml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Alunos: Nomes, Nomes**

Professores: Douglas Ribeiro e Franciane Rodrigues

## Preparação de Ambiente

In [1]:
# Instalaçao Bibliotecas
!pip install gcsfs -q

In [2]:
# Importando Bibliotecas
import os
import pandas as pd

from google.cloud import storage
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [3]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

### Configuração PySpark

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -N -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [6]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável

import pyspark.sql.functions as F
from pyspark.sql.types import *

spark

### Compartilhamento Chave GDrive

In [7]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada

target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


### Configuração da bucket

In [8]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [9]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket('projeto-final-ad2-e8')

### Conjuntos de dados

In [10]:
# Caminho do arquivo
path = 'gs://projeto-final-ad2-e8/dados/brutos/excel/iea_ponto_recarga_bruto.xlsx'

### Funções

In [11]:
# função de leitura no Spark
def spark_excel(path):
  try:

    try:
      # leitura csv com pandas
      read = pd.read_excel(path)
      # conversão em df pandas em df pyspark
      df = spark.createDataFrame(read)
      # êxito da função retornando o dataframe
      return df

    except:
      # leitura csv com pandas
      read = pd.read_excel(path)

      # tratamento de tipo de dado do df pandas para conversão em df pyspark
      for n in read.columns:
        if read[f'{n}'].dtype == object:
          read[f'{n}'] = read[f'{n}'].astype(str)
        elif read[f'{n}'].dtype == bool:
          read[f'{n}'] = read[f'{n}'].astype(bool)
        else:
          read[f'{n}'] = read[f'{n}'].astype(np.double)

      # conversão df pandas em df pyspark
      df = spark.createDataFrame(read)
      # êxito da função retornando o dataframe
      return df

  except:
    # indicação de erro na leitura do dataframe
    print(f'Falha na leitura do DataFrame: {path}!')

In [12]:
# função de definição do Schema
def docSchema(nome_dataset, dataframe, formato):
  '''Após a criação do schema e preenchimento das informações em um editor
  de texto, passar o conteúdo para validar no site: https://www.yamllint.com/'''
# modelo de impressão dos dados
  print(f"""
nome: {nome_dataset}
descricao:
origem:
tratamento: squadOito
armazenamento: GCS
acesso: privado
formato: {formato}
schema:""")
# laço de repetição para todas as colunas do dataframe
  for columns in dataframe.dtypes:
    print(f""" - coluna: {columns[0]}
   descricao:
   tipo: {columns[1]}""")

## Documentação

### Leitura do dataframe

In [13]:
df = spark_excel(path)

### Chamando função

In [14]:
docSchema('iea_ponto_recarga_bruto', df, 'xlsx')


nome: iea_ponto_recarga_bruto
descricao:
origem:
tratamento: squadOito
armazenamento: GCS
acesso: privado
formato: xlsx
schema:
 - coluna: region
   descricao:
   tipo: string
 - coluna: category
   descricao:
   tipo: string
 - coluna: parameter
   descricao:
   tipo: string
 - coluna: mode
   descricao:
   tipo: string
 - coluna: powertrain
   descricao:
   tipo: string
 - coluna: year
   descricao:
   tipo: bigint
 - coluna: unit
   descricao:
   tipo: string
 - coluna: value
   descricao:
   tipo: bigint
